# Tutorial 1.3: Introduction to Tracing

![](images/4_Notebook-13-Introduction-to-Tracing.png)

## Observability for GenAI Applications

Welcome to the tracing section! This is where MLflow really shines for GenAI development. You'll learn how to gain complete visibility into your LLM applications.

### What You'll Learn
- What is tracing and why it matters for GenAI
- Understanding the trace data model (traces, spans, hierarchy)
- Automatic tracing with autologging
- Tracing multiple frameworks (OpenAI, LangChain)
- Viewing and analyzing traces in MLflow UI
- Debugging with traces

### Prerequisites
- Completed Notebooks 1.1 and 1.2
- MLflow UI running
- OpenAI API key configured
- Or keys for your foundational model provider, e.g., Databricks

### Estimated Time: 30-35 minutes

---
## Step 1: What is Tracing?

### The Problem

Traditional experimenting tracking or logging isn't enough for LLM applications:

```python
# Traditional logging - hard to debug
print("Calling LLM...")
response = llm.generate(prompt)
print(f"Response: {response}")
# What happened inside? How long did it take? What was sent or received?
```

### The Solution: Distributed Tracing

Tracing captures the **complete execution flow** of your application:

```
Trace: RAG Application
├── Span 1: Embed Query [0ms - 200ms]
│   Input: "What is MLflow?"
│   Output: [0.123, 0.456, ...]
│   
├── Span 2: Retrieve Documents [200ms - 350ms]
│   Input: [0.123, 0.456, ...]
│   Output: ["MLflow is...", "The platform..."]
│   
└── Span 3: Generate Response [350ms - 1500ms]
    Input: {query, documents}
    Output: "MLflow is an open source platform..."
    LLM: gpt-5-mini
    Tokens: 150
```

### Key Benefits

1. **Visibility**: See every step in your LLM workflow
2. **Performance**: Identify bottlenecks and latency issues
3. **Debugging**: Trace errors to their exact source
4. **Cost**: Track token usage per operation
5. **Quality**: Inspect inputs/outputs at each step

---
## Step 2: Understanding the Trace Data Model

### Core Concepts

**Trace**: A complete execution of an operation
- Represents one request or workflow
- Contains one or more spans
- Has a root span

**Span**: A single operation within a trace
- Has a start and end time
- Contains inputs and outputs
- Has metadata (model, tokens, etc.)
- Can have parent-child relationships

**Span Attributes**: Additional metadata
- Model name
- Token counts
- Temperature
- Custom attributes

### Span Types

MLflow defines standard span types:

- `CHAIN`: A sequence of operations
- `LLM`: Language model call
- `RETRIEVER`: Document retrieval
- `EMBEDDING`: Text embedding
- `TOOL`: Tool/function execution
- `AGENT`: Agent reasoning
- `PARSER`: Output parsing

### Hierarchy Example

```
TRACE (root)
│
└─ SPAN: Agent Executor (AGENT)
   │
   ├─ SPAN: Planning Step (LLM)
   │  └─ attributes: {model: gpt-5, tokens: 50}
   │
   ├─ SPAN: Tool Execution (TOOL)
   │  └─ attributes: {tool: search, query: "..."}
   │
   └─ SPAN: Final Response (LLM)
      └─ attributes: {model: gpt-5, tokens: 150}
```

---
## Step 3: Environment Setup

In [1]:
import time

import mlflow
from dotenv import load_dotenv
from utils.clnt_utils import is_databricks_ai_gateway_client, get_databricks_ai_gateway_client, get_openai_client, get_ai_gateway_model_names

# Load environment
load_dotenv()

# Configure MLflow
mlflow.set_tracking_uri("http://localhost:5000")

use_databricks_provider = is_databricks_ai_gateway_client()
if use_databricks_provider:
    client = get_databricks_ai_gateway_client()
    model_name = get_ai_gateway_model_names()[0]
else:
    # Initialize OpenAI
    client = get_openai_client()
    model_name = "gpt-5-2"

print("✅ Environment configured: using", "Databricks" if use_databricks_provider else "OpenAI", "client")
print(f"   MLflow version: {mlflow.__version__}")
print(f"   Tracking URI: {mlflow.get_tracking_uri()}")
print(f"   Using model: {model_name}")

✅ Environment configured: using Databricks client
   MLflow version: 3.9.0
   Tracking URI: http://localhost:5000
   Using model: jsd-gpt-5-2


---
## Step 4: Your First Trace - Manual Example

Before we use autologging, let's understand what tracing captures by manually logging the information. This way we understand the difference and benefits of using autlogging.

In [2]:
# Create experiment for tracing examples
mlflow.set_experiment("06-tracing-introduction")

# Without tracing - basic call
print("\n📝 Making LLM call WITHOUT tracing...\n")

prompt = "Explain what distributed tracing is in one sentence."
response = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": prompt}],
    temperature=1.0,
    max_tokens=100
)

print(f"Response: {response.choices[0].message.content}")
print("\n❌ What we DON'T know:")
print("   - Exact timing of the call")
print("   - Detailed request/response structure")
print("   - Easy way to correlate with other operations")
print("   - Visual representation of execution")


📝 Making LLM call WITHOUT tracing...

Response: Distributed tracing is a way to track and correlate a single request as it flows through multiple services and components in a distributed system, recording timing and metadata for each step to help diagnose performance issues and failures.

❌ What we DON'T know:
   - Exact timing of the call
   - Detailed request/response structure
   - Easy way to correlate with other operations
   - Visual representation of execution


---
## Step 5: Automatic Tracing with OpenAI Autologging

Now let's enable tracing with a single line of code!

In [3]:
# Enable OpenAI autologging - THIS IS THE MAGIC LINE!
mlflow.openai.autolog()

print("✅ OpenAI autologging enabled")
print("   All OpenAI API calls will now be automatically traced!")

✅ OpenAI autologging enabled
   All OpenAI API calls will now be automatically traced!


In [5]:
# Make the same call - now it's automatically traced!
print("\n🔍 Making LLM call WITH tracing...\n")

response = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": prompt}],
    temperature=1.0,
    max_tokens=100
)

print(f"Response: {response.choices[0].message.content}")
print("\n✅ What we NOW know:")
print("   ✓ Complete request details (model, messages, parameters)")
print("   ✓ Response content and metadata")
print("   ✓ Token usage (prompt, completion, total)")
print("   ✓ Timing information (latency)")
print("   ✓ All captured automatically!")
print("\n🔗 View trace in MLflow UI: http://localhost:5000")
print("   Navigate to: Traces tab")


🔍 Making LLM call WITH tracing...

Response: Distributed tracing is a way to track and visualize a single request as it flows through multiple services and components in a distributed system, linking related operations together to diagnose latency and failures.

✅ What we NOW know:
   ✓ Complete request details (model, messages, parameters)
   ✓ Response content and metadata
   ✓ Token usage (prompt, completion, total)
   ✓ Timing information (latency)
   ✓ All captured automatically!

🔗 View trace in MLflow UI: http://localhost:5000
   Navigate to: Traces tab


Trace(trace_id=tr-6e7dc2a97971aff86ceb1e3513d234f7)

### 🎉 What Just Happened?

With `mlflow.openai.autolog()`, MLflow automatically:

1. **Intercepted** the OpenAI API call
2. **Created** a trace with a unique ID
3. **Captured** all inputs (messages, model, parameters)
4. **Captured** all outputs (response, tokens, timing)
5. **Stored** everything in a structured format
6. **Made it available** in the MLflow UI

**Zero code changes required!**

---
## Step 6: Exploring Trace Details

Let's make a more complex call and examine what gets traced.

In [6]:
# Multi-turn conversation with system prompt
print("\n💬 Multi-turn conversation with tracing...\n")

messages = [
    {
        "role": "system",
        "content": "You are a helpful AI expert who explains concepts clearly and concisely."
    },
    {
        "role": "user",
        "content": "What is MLflow?"
    },
    {
        "role": "assistant",
        "content": "MLflow is an open-source platform for managing the machine learning lifecycle."
    },
    {
        "role": "user",
        "content": "What are its main components and what are they used for?"
    }
]

response = client.chat.completions.create(
    model=model_name,
    messages=messages,
    temperature=1.0,
    max_tokens=200
)

print(f"Response: {response.choices[0].message.content[:200]}...")
print("\n📊 Trace captured:")
print(f"   Model: {response.model}")
print(f"   Prompt tokens: {response.usage.prompt_tokens}")
print(f"   Completion tokens: {response.usage.completion_tokens}")
print(f"   Total tokens: {response.usage.total_tokens}")
print("\n🔍 In the trace, you can see:")
print("   - All 4 messages (system + conversation history)")
print("   - Model configuration (temperature, max_tokens)")
print("   - Complete response")
print("   - Token breakdown")


💬 Multi-turn conversation with tracing...

Response: ...

📊 Trace captured:
   Model: gpt-5.2-2025-12-11
   Prompt tokens: 66
   Completion tokens: 200
   Total tokens: 266

🔍 In the trace, you can see:
   - All 4 messages (system + conversation history)
   - Model configuration (temperature, max_tokens)
   - Complete response
   - Token breakdown


Trace(trace_id=tr-f315b641fc7048a87050841ed9b5f1d5)

---
## Step 7: Tracing Multiple Sequential Calls

Let's see how tracing helps with multi-step workflows.

In [7]:
# Simple multi-step workflow
print("\n🔄 Multi-step workflow with automatic tracing...\n")

# Step 1: Generate a topic
print("Step 1: Generating topic...")
topic_response = client.chat.completions.create(
    model=model_name,
    messages=[
        {
        "role": "system",
            "content": "You are a helpful AI expert who explains concepts clearly and concisely."
        },
        {
        "role": "user",
        "content": "Suggest one interesting AI topic for a blog post."
    }],
    temperature=1.0,
    max_tokens=200
)
topic = topic_response.choices[0].message.content
print(f"  Topic: {topic}")

# Step 2: Generate an outline
print("\nStep 2: Creating outline...")
outline_response = client.chat.completions.create(
    model=model_name,
    messages=[{
        "role": "user",
        "content": f"Create a 3-point outline for a blog post about: {topic}"
    }],
    temperature=1.0,
    max_tokens=500
)
outline = outline_response.choices[0].message.content
print(f"  Outline: {outline[:100]}...")

# Step 3: Write the introduction
print("\nStep 3: Writing introduction...")
intro_response = client.chat.completions.create(
    model=model_name,
    messages=[{
        "role": "user",
        "content": f"Write a 4-sentence introduction paragraph for: {topic}"
    }],
    temperature=1.0,
    max_tokens=2000
)
intro = intro_response.choices[0].message.content
print(f"  Introduction: {intro}")

print("\n✅ All three steps completed!")
print("\n📊 Total tokens used:")
total_tokens = (topic_response.usage.total_tokens +
                outline_response.usage.total_tokens +
                intro_response.usage.total_tokens)
print(f"   {total_tokens} tokens")

print("\n🔍 View in MLflow UI:")
print("   You'll see THREE separate traces, one for each call")
print("   Each trace shows timing, tokens, and complete I/O")


🔄 Multi-step workflow with automatic tracing...

Step 1: Generating topic...
  Topic: **Topic:** *AI Agents That Use Tools (Beyond Chatbots): How “LLM + Tools” Systems Work*

Write about how modern AI goes beyond generating text by **calling external tools**—like web search, databases, calculators, code execution, calendars, or APIs—to complete tasks. Cover:

- What an “AI agent” is vs. a standard chatbot  
- How tool use works (function calling, planning, feedback loops)  
- Real-world examples (customer support automation, data analysis, scheduling)  
- Key risks and design challenges (hallucinations, permissions, security, reliability)  
- Where this trend is headed (multi-agent systems, long-term memory, autonomy limits)

Step 2: Creating outline...
  Outline: 1) **From Chatbots to AI Agents: What Changes and Why It Matters**  
   - Define a *standard chatbot...

Step 3: Writing introduction...
  Introduction: Modern AI is increasingly moving beyond standard chatbots that only gen

[Trace(trace_id=tr-12994f2cd5b56d61ccb159750daf0bfa), Trace(trace_id=tr-1c7e18e0a79f24e2b0d603b99cf5f71d), Trace(trace_id=tr-5f0abe6d85b4816d2e3bbadbad453f1e)]

### 💡 Observation

Each OpenAI call creates a **separate trace**. In the next notebook, we'll learn how to group related calls into a **single hierarchical trace** using manual instrumentation.

---
## Step 8: Tracing with Different Models

Let's compare traces across different models.

In [8]:
# Compare different models
prompt = "Explain LLM temperature in 20 words or less."
models = ["jsd-gpt-5-2", "jsd-gpt-5-mini"] if use_databricks_provider else ["gpt-5-2", "gpt-5-mini"]


print("\n🔬 Comparing models with tracing...\n")

for model in models:
    print(f"Testing {model}...")
    
    start = time.time()
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=1.0,
        max_tokens=1000
    )
    latency = time.time() - start
    
    print(f"  Latency: {latency:.2f}s")
    print(f"  Tokens: {response.usage.total_tokens}")
    print(f"  Response: {response.choices[0].message.content}\n")

print("✅ Model comparison complete!")
print("\n🔍 In MLflow UI Traces tab:")
print("   - Filter by Execution time (ms)")
print("   - Compare latencies")


🔬 Comparing models with tracing...

Testing jsd-gpt-5-2...
  Latency: 1.07s
  Tokens: 42
  Response: LLM temperature controls randomness: low = more deterministic, high = more varied and creative but less predictable outputs.

Testing jsd-gpt-5-mini...
  Latency: 1.26s
  Tokens: 49
  Response: Temperature controls randomness in an LLM's token sampling: low = deterministic, high = more diverse and unpredictable outputs.

✅ Model comparison complete!

🔍 In MLflow UI Traces tab:
   - Filter by Execution time (ms)
   - Compare latencies


[Trace(trace_id=tr-47456ea6c54d0d92dd2fbbc92845b5c1), Trace(trace_id=tr-c91678e54408c7cc6d2b939da10e63fb)]

---
## Step 9: Understanding the MLflow Traces UI

Let's explore what the Traces tab shows you.

In [9]:
# Generate a trace with interesting attributes
print("""
╔══════════════════════════════════════════════════════════════╗
║           MLflow Traces UI Guide                             ║
╚══════════════════════════════════════════════════════════════╝

1. 🔍 TRACES TAB LOCATION:
   Navigate to: http://localhost:5000
   Click on: "Traces" tab (Under Observability)

2. 📋 TRACE LIST VIEW:
   - Timestamp: When the trace was created
   - Request ID: Unique identifier
   - Latency: Total execution time
   - Status: Success/Error
   - Tags: Custom metadata

3. 📊 TRACE DETAILS VIEW (click on a trace):
   
   Timeline Tab:
   - Waterfall visualization of spans
   - Duration of each operation
   - Parent-child relationships
   
   Spans Tab:
   - List of all spans in the trace
   - Inputs and outputs for each span
   - Metadata (model, tokens, etc.)
   
   Attributes Tab:
   - Custom attributes you've logged
   - System attributes (timestamps, status)

4. 🔎 SPAN DETAILS (click on a span):
   - Input: What was sent to the LLM
   - Output: What the LLM returned
   - Attributes: Model config, tokens, etc.
   - Timing: Start time, end time, duration

5. 💡 USEFUL FEATURES:
   - Search traces by content
   - Filter by date range
   - Filter by status (success/error)
   - Compare multiple traces
   - Export trace data

""")

# Make a traced call for demonstration
response = client.chat.completions.create(
    model=model_name,
    messages=[{
        "role": "user",
        "content": "Explain the MLflow Traces UI in one sentence."
    }],
    temperature=1.0,
    max_tokens=1000
)

print("✅ Trace created! Go to the UI and explore it.")


╔══════════════════════════════════════════════════════════════╗
║           MLflow Traces UI Guide                             ║
╚══════════════════════════════════════════════════════════════╝

1. 🔍 TRACES TAB LOCATION:
   Navigate to: http://localhost:5000
   Click on: "Traces" tab (Under Observability)

2. 📋 TRACE LIST VIEW:
   - Timestamp: When the trace was created
   - Request ID: Unique identifier
   - Latency: Total execution time
   - Status: Success/Error
   - Tags: Custom metadata

3. 📊 TRACE DETAILS VIEW (click on a trace):

   Timeline Tab:
   - Waterfall visualization of spans
   - Duration of each operation
   - Parent-child relationships

   Spans Tab:
   - List of all spans in the trace
   - Inputs and outputs for each span
   - Metadata (model, tokens, etc.)

   Attributes Tab:
   - Custom attributes you've logged
   - System attributes (timestamps, status)

4. 🔎 SPAN DETAILS (click on a span):
   - Input: What was sent to the LLM
   - Output: What the LLM returned


Trace(trace_id=tr-2996e88a5115dac8270553a2ffed862b)

---
## Step 10: Tracing with LangChain (Framework Integration)

MLflow supports automatic tracing for many frameworks. Let's try LangChain!

LangChain is a Python/JS framework for building LLM-powered applications by providing reusable building blocks for prompts, models, tools/function-calling, retrieval (RAG), memory/state, and multi-step chains/agents.

Why/when to use it
Use LangChain when you want to:

**Standardize an LLM app**: consistent interfaces for prompt templates, chat models (e.g., ChatOpenAI), outputs/parsers, retries, and tracing.

**Build RAG**: connect documents → embeddings → vector store → retriever → prompt, with common patterns already implemented.

**Use tools/agents**: let the model call functions/tools and orchestrate multi-step workflows.

**Compose pipelines**: easily chain steps (prompt → model → parser) and swap components without rewriting glue code.

In [ ]:
# Install LangChain if needed
!pip install langchain langchain-openai -q

In [10]:
from langchain_core.prompts import ChatPromptTemplate

databricks_provider = is_databricks_ai_gateway_client()
# Enable LangChain autologging
mlflow.langchain.autolog()

print("✅ LangChain autologging enabled")
print("✅ Using", "Databricks AI Gateway" if databricks_provider else "OpenAI", "as provider")

✅ LangChain autologging enabled
✅ Using Databricks AI Gateway as provider


In [11]:
from utils.clnt_utils import get_langchain_chat_openai_client, get_databricks_ai_gateway_langchain_client

# Create a simple LangChain chain
print("\n🔗 Creating and running LangChain chain with tracing...\n")

# Define prompt template
prompt_template = ChatPromptTemplate.from_template(
    "You are a {role}. Answer the following question: {question}"
)

# Create LLM
if databricks_provider:
    llm = get_databricks_ai_gateway_langchain_client(model_name, temperature=1.0)
else:
    llm = get_langchain_chat_openai_client(mode=model_name, temperature=1.0)

# Create chain
chain = prompt_template | llm

# Run chain
response = chain.invoke({
    "role": "helpful AI assistant",
    "question": "What are the benefits of tracing in GenAI applications?"
})

print(f"Response: {response.content}")
print("\n✅ LangChain execution traced!")
print("\n🔍 In the trace, you'll see:")
print("   - Prompt template construction")
print("   - Variable substitution")
print("   - LLM invocation")
print("   - All as separate spans in a hierarchy!")


🔗 Creating and running LangChain chain with tracing...

Response: Tracing in GenAI applications (often via distributed tracing/telemetry) means recording structured “spans” for each step of a request—prompt construction, retrieval, model calls, tool/function calls, parsing, and post-processing—so you can see exactly what happened and why. Key benefits:

- **End-to-end observability**: See the full path of a user request across services (API, RAG pipeline, LLM, tools) with timing and metadata.
- **Faster debugging & RCA**: Pinpoint where failures occur (bad prompt, retrieval miss, tool error, parsing bug) and reproduce issues with the exact inputs/outputs.
- **Performance & latency optimization**: Identify bottlenecks (slow vector search, long model latency, tool delays), measure tokenization/streaming effects, and reduce tail latency.
- **Quality improvement**: Correlate output quality with prompt versions, retrieved documents, tool choices, and model parameters; detect where hallucin

Trace(trace_id=tr-ff4f224c7ebd24c77024da091f96b220)

### 🎯 LangChain Tracing Benefits

With LangChain autologging, MLflow automatically traces:

- **Chain execution**: See how data flows through the chain
- **Prompt construction**: View how templates are filled
- **LLM calls**: Standard OpenAI tracing
- **Retrieval steps**: If using RAG components
- **Tool usage**: If using agents with tools

**All without manual instrumentation!**

---
## Step 11: Error Tracing and Debugging

Tracing is especially valuable when things go wrong.

In [12]:
# Intentionally cause an error to see how it's traced
print("\n🐛 Demonstrating error tracing...\n")

try:
    # This will fail - invalid model name
    response = client.chat.completions.create(
        model="gpt-does-not-exist",
        messages=[{"role": "user", "content": "Hello"}]
    )
except Exception as e:
    print(f"❌ Error occurred: {type(e).__name__}")
    print(f"   Message: {str(e)[:100]}...")
    print("\n✅ Error was captured in trace!")
    print("\n🔍 In the MLflow UI:")
    print("   - Trace marked with error status")
    print("   - Error message visible")
    print("   - Stack trace preserved")
    print("   - Easy to identify failure point")


🐛 Demonstrating error tracing...

❌ Error occurred: NotFoundError
   Message: Error code: 404 - {'error_code': 'NOT_FOUND', 'message': "AI Gateway endpoint 'gpt-does-not-exist' d...

✅ Error was captured in trace!

🔍 In the MLflow UI:
   - Trace marked with error status
   - Error message visible
   - Stack trace preserved
   - Easy to identify failure point


Trace(trace_id=tr-5774aacae296e2d051497c1613a13605)

### 🐛 Debugging with Traces

When your LLM application fails:

1. **Find the trace** in the UI (filter by error status)
2. **Identify the failing span** (marked red)
3. **Inspect inputs** that caused the error
4. **View error details** (message, type, stack trace)
5. **Fix and re-run** with full history preserved

---
## Step 12: Disabling Autologging

Sometimes you want to turn tracing off temporarily.

In [13]:
# Disable autologging
mlflow.openai.autolog(disable=True)
mlflow.langchain.autolog(disable=True)

print("❌ Autologging disabled")

# This call won't be traced
response = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": "This won't be traced."}]
)

print("\n✅ Call completed without tracing")

# Re-enable for future calls
mlflow.openai.autolog()
print("\n✅ Autologging re-enabled")

❌ Autologging disabled

✅ Call completed without tracing

✅ Autologging re-enabled


---
## Step 13: Best Practices for Tracing

Let's review key best practices.

In [14]:
print("""
╔══════════════════════════════════════════════════════════════╗
║           Tracing Best Practices                             ║
╚══════════════════════════════════════════════════════════════╝

✅ DO:

1. Enable autologging early in development
   - Easier to debug from the start
   - Understand performance characteristics early

2. Use tracing in all environments
   - Development: Full tracing for debugging
   - Staging: Trace all requests
   - Production: Sample or trace all (depending on volume)

3. Combine tracing with experiment tracking
   - Traces give you the "how"
   - Experiments give you the "what" and "why"

4. Review traces regularly
   - Look for performance bottlenecks
   - Identify expensive operations
   - Understand failure patterns

5. Use multiple frameworks
   - MLflow supports 30+ integrations
   - Pick the best tool for each task

❌ DON'T:

1. Ignore traces until there's a problem
   - Proactive monitoring prevents issues

2. Log sensitive data in traces
   - PII, credentials, confidential info
   - Use data masking if needed

3. Assume autologging captures everything
   - Custom operations need manual instrumentation
   - We'll cover this in the next notebook!

4. Over-rely on individual traces
   - Look at aggregate patterns
   - Use traces + metrics together

💡 PRO TIPS:

- Set up trace sampling in high-volume production
- Archive old traces to manage storage
- Export important traces for documentation
- Share trace links with your team for debugging

""")


╔══════════════════════════════════════════════════════════════╗
║           Tracing Best Practices                             ║
╚══════════════════════════════════════════════════════════════╝

✅ DO:

1. Enable autologging early in development
   - Easier to debug from the start
   - Understand performance characteristics early

2. Use tracing in all environments
   - Development: Full tracing for debugging
   - Staging: Trace all requests
   - Production: Sample or trace all (depending on volume)

3. Combine tracing with experiment tracking
   - Traces give you the "how"
   - Experiments give you the "what" and "why"

4. Review traces regularly
   - Look for performance bottlenecks
   - Identify expensive operations
   - Understand failure patterns

5. Use multiple frameworks
   - MLflow supports 30+ integrations
   - Pick the best tool for each task

❌ DON'T:

1. Ignore traces until there's a problem
   - Proactive monitoring prevents issues

2. Log sensitive data in traces
   - P

---
## Summary

In this notebook, you learned:

1. ✅ What tracing is and why it's critical for GenAI
2. ✅ The trace data model (traces, spans, hierarchy)
3. ✅ Automatic tracing with `mlflow.openai.autolog()`
4. ✅ How to view and analyze traces in the MLflow UI
5. ✅ Tracing multi-step workflows
6. ✅ Framework integration (LangChain example)
7. ✅ Error tracing and debugging
8. ✅ Best practices for production use

### Key Takeaways

- **One line of code** enables complete observability
- **Automatic capture** of inputs, outputs, and metadata
- **Framework agnostic** - works with OpenAI, LangChain, LlamaIndex, etc.
- **Essential for debugging** complex LLM workflows
- **Production-ready** with sampling and filtering

### What's Next?

**📓 Notebook 1.4: Manual Tracing and Advanced Observability**

Learn how to:
- Create custom spans with `@mlflow.trace`
- Add custom attributes to spans
- Build hierarchical traces for complex workflows
- Trace RAG pipelines with retrieval and generation steps
- Trace agentic workflows with tool usage
- Debug production issues with advanced techniques

### 🎯 Practice Exercise

Before moving on, try this:

1. Enable tracing
2. Build a simple chatbot that:
   - Takes user input
   - Generates a response
   - Evaluates the response quality
3. View all three operations in a single trace
4. Compare performance across multiple runs

Ready? Let's continue to manual tracing!